In [1]:
import pandas as pd 
import json
import os

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Preparando dicionário para o DataFrame

In [2]:
rows = {}
rows['Evolucao'] = []
rows['Target'] = []
with open('layer_fatores_queda.json') as jfile:
    data = json.load(jfile)
    for tag in data['tags']: 
        rows[tag['tag_name']] = [] 

## Preparando as anotações

In [3]:
def create_values_tags():
    values_tags = {}
    with open('layer_fatores_queda.json') as jfile:
        data = json.load(jfile)
        for tag in data['tags']: 
            values_tags[tag['tag_name']] = 0
    return values_tags

In [4]:
def get_annotated_sentences(text, annotated_s):
    annotated_sentences = [] 
    for annotated in annotated_s:
        try:
            if text[annotated['begin']:annotated['end']][-1:] == '.':
                annotated_sentences.append((text[annotated['begin']:annotated['end']][:-1], annotated['value']))
            else:
                annotated_sentences.append((text[annotated['begin']:annotated['end']], annotated['value']))
        except KeyError:
            continue
    return annotated_sentences

In [5]:
def extract_clinical_notes(text): 
    clinical_notes = []
    text_ = text.split('.\n')
    for line in text_[7:]:
        if line[:9] == 'Evolução:':
           clinical_notes.append(line[9:]) 
    return clinical_notes

In [6]:
def has_fall(value_tags):
    lst = [v for k,v in value_tags.items() if 'Tipo de Queda' in k]
    return 1 if 1 in lst else 0

In [7]:
def read_json(subdir,file):    
    with open(subdir + "/" + file) as jfile:
        data = json.load(jfile)
        text = data['_referenced_fss']['12']['sofaString']
        file = subdir.split('/')[2]
        clinical_notes = extract_clinical_notes(text)
        annotated_sentences = get_annotated_sentences(text, data['_views']['_InitialView']['Teste'])
        for clinical_note in clinical_notes:
            remove_sentences = []
            values_tags = create_values_tags()
            rows['Evolucao'].append(clinical_note)
            for annotated in annotated_sentences:
                if annotated[0] in clinical_note:
                    remove_sentences.append(annotated)                
                    try:
                        values_tags[annotated[1]] = 1 
                    except KeyError:
                        pass
            rows['Target'].append(has_fall(values_tags))
            annotated_sentences = [x for x in annotated_sentences if x not in remove_sentences]
            for key, value in values_tags.items():
                rows[key].append(value)

## Percorrendo os arquivos das anotações

In [8]:
rootdir = './annotated_clinical_notes'
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if (os.path.join(subdir, file)[-11:] == 'amanda.json'):
            read_json(subdir, file)

In [9]:
df = pd.DataFrame.from_dict(rows, orient='index')
df = df.transpose()

In [11]:
df.to_csv('training_data.csv')